<a href="https://colab.research.google.com/github/azzindani/03_Prompt_Engineering/blob/main/PE_EN_Qwen2_5_0_5B_4bit_Med_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Modules

In [1]:
!pip install -U -q bitsandbytes
!pip install -q optimum
!pip install -q auto-gptq

In [2]:
import torch

from transformers import (
  AutoTokenizer,
  AutoModelForCausalLM,
  GenerationConfig,
  BitsAndBytesConfig,
  GPTQConfig
)

if torch.cuda.is_available():
  print('GPU is available!')
else:
  print('GPU is not available.')

GPU is available!


## Import Model

In [3]:
model_name = 'Qwen/Qwen2.5-0.5B'

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit = True,
  bnb_4bit_quant_type = 'nf4',
  bnb_4bit_compute_dtype = torch.float16,
  bnb_4bit_use_double_quant = True,
)

model = AutoModelForCausalLM.from_pretrained(
  model_name,
  quantization_config = bnb_config,
  trust_remote_code = True
).to(device) #'''

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [6]:
'''model = AutoModelForCausalLM.from_pretrained(
  model_name,
  torch_dtype = torch.float16,
  trust_remote_code = True
).to(device) #'''

'model = AutoModelForCausalLM.from_pretrained(\n  model_name,\n  torch_dtype = torch.float16,\n  trust_remote_code = True\n).to(device) #'

## Import Tokenizer

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

## Model Preview

In [8]:
def assistant(prompt):
  prompt_format = f"### Human: {prompt}\n### Assistant :"

  inputs = tokenizer(
    prompt_format,
    return_tensors = 'pt'
  ).to(device)
  generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = 0.1,
    max_new_tokens = 1024,
    pad_token_id = tokenizer.eos_token_id
  )
  outputs = model.generate(
    **inputs,
    generation_config = generation_config
  )
  return print(tokenizer.decode(outputs[0], skip_special_tokens = True))

In [9]:
# Level 1: Basic Prompt
prompt = "What are the symptoms and treatments of diabetes?"
assistant(prompt)

### Human: What are the symptoms and treatments of diabetes?
### Assistant : Diabetes is a chronic disease that affects the body's ability to regulate blood sugar levels. It is caused by the body producing insulin, which is a hormone that helps the body cells absorb sugar from the blood. Diabetes can cause a range of symptoms, including increased thirst, frequent urination, fatigue, and unexplained weight loss. There are also treatments for diabetes, including diet, exercise, and medication.


In [10]:
# Level 2: Adding Detail
prompt = "Briefly describe the symptoms and treatments of diabetes."
assistant(prompt)

### Human: Briefly describe the symptoms and treatments of diabetes.
### Assistant : Diabetes is a chronic disease that affects the body's ability to regulate blood sugar levels. It is caused by the body's inability to produce enough insulin or the body's cells are unable to use insulin effectively. The symptoms of diabetes include frequent urination, increased thirst, fatigue, and unexplained weight loss. The treatments of diabetes include lifestyle changes such as diet and exercise, medication to control blood sugar levels, and monitoring blood sugar levels regularly.


In [11]:
# Level 3: Specify Information Type and Audience
prompt = "Explain the common symptoms and treatments of diabetes to a general audience."
assistant(prompt)

### Human: Explain the common symptoms and treatments of diabetes to a general audience.
### Assistant : Diabetes is a chronic condition that affects the body's ability to regulate blood sugar levels. It is a common disease that affects people of all ages, but it is more prevalent in certain groups such as people with a family history of diabetes, people who are overweight or obese, and people who have a sedentary lifestyle. The common symptoms of diabetes include increased thirst, frequent urination, fatigue, and blurry vision. The treatments of diabetes include lifestyle changes such as diet and exercise, medication such as insulin or metformin, and monitoring blood sugar levels with a glucometer.


In [12]:
# Level 4: Request for Structured Response
prompt = "List the main symptoms of diabetes, then outline the standard treatments."
assistant(prompt)

### Human: List the main symptoms of diabetes, then outline the standard treatments.
### Assistant : Diabetes is a chronic disease that affects the body's ability to regulate blood sugar levels. The main symptoms of diabetes include frequent urination, increased thirst, fatigue, and unexplained weight loss. The standard treatments for diabetes include diet, exercise, and medication.


In [13]:
# Level 5: Differentiate by Type
prompt = "List the symptoms and treatments for both Type 1 and Type 2 diabetes."
assistant(prompt)

### Human: List the symptoms and treatments for both Type 1 and Type 2 diabetes.
### Assistant : Type 1 diabetes is a condition where the body's immune system attacks and destroys the insulin-producing cells in the pancreas, leading to a lack of insulin. Symptoms include frequent infections, slow healing of cuts and bruises, and increased thirst and hunger. Treatment includes insulin injections, lifestyle changes, and dietary adjustments. Type 2 diabetes is a condition where the body's cells do not respond to insulin, leading to a lack of glucose in the blood. Symptoms include fatigue, weight loss, increased thirst and hunger, and increased urination. Treatment includes lifestyle changes, diet changes, and medication.


In [14]:
# Level 6: Target a Professional Audience
prompt = "Describe the symptoms, diagnostic criteria, and treatment options for Type 1 and Type 2 diabetes, focusing on what healthcare providers should know."
assistant(prompt)

### Human: Describe the symptoms, diagnostic criteria, and treatment options for Type 1 and Type 2 diabetes, focusing on what healthcare providers should know.
### Assistant : Type 1 and Type 2 diabetes are both chronic conditions that require medical attention. They are caused by a lack of insulin in the body, which leads to high blood sugar levels. The symptoms of both types of diabetes include frequent urination, increased thirst, fatigue, and increased appetite. The diagnostic criteria for Type 1 diabetes include symptoms such as sudden weight loss, increased thirst, increased urination, and increased appetite. Treatment options for Type 1 diabetes include insulin injections or insulin pumps, while treatment options for Type 2 diabetes include diet, exercise, and medication. Healthcare providers should know about the symptoms, diagnostic criteria, and treatment options for both types of diabetes to provide appropriate care and management.


In [15]:
# Level 7: Clinical Detail with Disease Stages
prompt = "Explain the symptoms, diagnostic markers, and treatments for diabetes, distinguishing between prediabetes, Type 1, and Type 2 diabetes. Include early signs, progression, and advanced stages."
assistant(prompt)

### Human: Explain the symptoms, diagnostic markers, and treatments for diabetes, distinguishing between prediabetes, Type 1, and Type 2 diabetes. Include early signs, progression, and advanced stages.
### Assistant : To explain the symptoms, diagnostic markers, and treatments for diabetes, we will focus on Type 2 diabetes, which is the most common form of diabetes. Type 2 diabetes is characterized by the presence of insulin resistance, which leads to a lack of insulin production or impaired insulin action. The symptoms of Type 2 diabetes can include fatigue, weight loss, increased thirst and urination, increased appetite, and blurred vision. The diagnostic markers for Type 2 diabetes include blood sugar levels, blood pressure, and body mass index (BMI). The treatments for Type 2 diabetes include lifestyle changes, medication, and insulin therapy. Early signs of Type 2 diabetes include increased thirst and urination, fatigue, weight loss, and increased appetite. The progression of Type

In [ ]:
# Level 8: Add Nuance with Treatment Personalization
prompt = "Describe the symptoms, diagnostic markers, and treatment options for diabetes, with a focus on Type 1 and Type 2. Include how treatments may vary based on patient age, co-existing conditions, and lifestyle factors."
assistant(prompt)

In [ ]:
# Level 9: Consider Risks, Complications, and Treatment Impacts
prompt = "Provide a comprehensive overview of diabetes, covering symptoms, diagnostic criteria, and treatment options for both Type 1 and Type 2 diabetes. Explain how risks and complications, such as neuropathy and cardiovascular disease, may impact treatment decisions. Include possible side effects and patient adherence concerns for each treatment."
assistant(prompt)

In [ ]:
# Level 10: Advanced Detail for Clinical Research or Specialist Training
prompt = "Provide an in-depth, evidence-based analysis of the symptoms, diagnostic criteria, and treatment approaches for diabetes, including both Type 1 and Type 2. Discuss the pathophysiology, biomarkers, and clinical guidelines, as well as how complications like neuropathy, retinopathy, and cardiovascular disease alter treatment. Include recent clinical research findings, new pharmaceutical options, and the role of lifestyle interventions in managing long-term patient outcomes."
assistant(prompt)